
# Fishman Maxim 

In [ ]:
import pandas as pd
!pip install researchpy
import researchpy as rp
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from scipy import stats
from scipy.interpolate import *
from matplotlib.pyplot import *
import numpy as np 
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
import statistics
from sklearn.metrics import f1_score
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_accept = pd.read_csv("13_accept.csv")
df_reject = pd.read_csv("13_reject.csv")
df_accept.drop('UID', axis=1, inplace=True)
df_reject.drop('UID', axis=1, inplace=True)

FileNotFoundError: ignored

## 1. Какая доля 1 в выборке "accept"?

In [ ]:
print("Доля 1:", df_accept['target'].mean())

## 2. Необходимо рассчитать для всех интервальных переменных следующее:
  - Доля пропущенных значений 
  - Медиана
  - Среднее
  - Среднеквадратическое отклонение
  - Есть ли аномальные значения, выбросы? 
  - Information Value

### 2.1 Accept

In [ ]:
df_accept.T

There are 8 interval variables:

"income", "COUNT_ACTIVE_CREDIT_NO_CC", "CNT_MNTH_FROM_LAST_PMNT", "age", "experience", "inquiry_14_day", "inquiry_21_day", "count_mnth_act_passport"

(After getting rid of Nan values I saw see that "DEPENDANT_NUMBER" variable has only 5 possible values => I will use it as categorical variable.) 

I will replace Nans with medians for discreet variables and with means for continuous variables.

In [ ]:
int_accept1 = pd.DataFrame(df_accept,columns=['income','COUNT_ACTIVE_CREDIT_NO_CC',
                                             'CNT_MNTH_FROM_LAST_PMNT','age', 'experience', 
                                             'inquiry_14_day','inquiry_21_day','count_mnth_act_passport'])
int_accept = int_accept1

In [ ]:
int_accept['CNT_MNTH_FROM_LAST_PMNT'] = int_accept['CNT_MNTH_FROM_LAST_PMNT'].fillna(int_accept["CNT_MNTH_FROM_LAST_PMNT"].mean())
int_accept['experience'] = int_accept['experience'].fillna(int_accept["experience"].mean())
int_accept['count_mnth_act_passport'] = int_accept['count_mnth_act_passport'].fillna(int_accept["count_mnth_act_passport"].mean())
int_accept['age'] = int_accept['age'].fillna(int_accept["age"].mean())

int_accept['income'] = int_accept['income'].fillna(int_accept["income"].median())
int_accept['COUNT_ACTIVE_CREDIT_NO_CC'] = int_accept['COUNT_ACTIVE_CREDIT_NO_CC'].fillna(int_accept["COUNT_ACTIVE_CREDIT_NO_CC"].median())
int_accept['inquiry_14_day'] = int_accept['inquiry_14_day'].fillna(int_accept["inquiry_14_day"].median())
int_accept['inquiry_21_day'] = int_accept['inquiry_21_day'].fillna(int_accept["inquiry_21_day"].median())

if(int_accept.isnull().values.any() == False):
    print("No more Nans")



Here, I will calculate percentage of Nan variables, mean, median and standard deviation. To check anomalies in the data I will use z-score method and I will count the number of outliers for each variable:

In [ ]:
percent_non = []
percent_exst = []
count_none = []
col_names = []
for col in int_accept1:
    col_names.append(col)
    percent_non.append((df_accept[col].isnull().sum())/(len(df_accept)/100))
    count_none.append(df_accept[col].isnull().sum())
    

median = []
mean = []
std = []
outliers = []


def z_score_method(df, variable_name):
    columns = df.columns
    z = np.abs(stats.zscore(df))
    threshold = 3
    outlier = []
    index=0
    for item in range(len(columns)):
        if columns[item] == variable_name:
            index = item
    for i, v in enumerate(z[:, index]):
        if v > threshold:
            outlier.append(i)
        else:
            continue
    return outlier
    
for col in int_accept:
    median.append(int_accept[col].median())
    mean.append(int_accept[col].mean())
    std.append(int_accept[col].std())
    outliers.append(len(z_score_method(int_accept, col)))

The following function calculates IV for each varible using WOE. The method of calculating WOE has two steps:

1) Fine classing: I separated data in 20 bins and calculate WOE value for each bin;

2) Coarse classing: In order to decrease number of bins I combine bins with close WOE values;

Then the function calculates IV for each bin and sums up values to get IV for the variable

In [ ]:
def get_IV(df, feature, target):
    lst = []
    bins = df[feature].quantile(np.linspace(0, 1, 21)).values
    bin_index = range(1, len(bins))
    for index in bin_index:
        val = df[(df[feature] >= bins[index-1])&(df[feature] <= bins[index])]
        lst.append([feature,               
                    val[(val[target] == 0)].count()[feature],  # Good 
                    val[(val[target] == 1)].count()[feature]   # Bad  
                   ])
 
    data = pd.DataFrame(lst, columns=['Variable', 'Good', 'Bad'])

        
    total_bad = df[df[target] == 1].count()[feature]
    total_good = df.shape[0] - total_bad
    
    data['Distribution Good'] = data['Good']/ total_good
    data['Distribution Bad'] = data['Bad'] / total_bad
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    data = data.sort_values(by=['WoE'], ascending=[True])
    data.index = range(len(data.index))
    
    data1 = data
    
        
    #Combine bins with similar WOE values
    n = range(1, len(data1))
    for i in n:
        if (data1['WoE'][i] - data1['WoE'][i-1] < 0.08):
            data.loc[i,'WoE'] = data1['WoE'][i-1]
    
    
    #Find IV for variable
    iv_all = []
    uniq = data['WoE'].unique()
    for val in uniq:
        b = data[(data['WoE'] == val)]
        b.index = range(len(b.index))
        good = b['Good'].sum()
        bad = b['Bad'].sum()
        ival = val * (good/total_good - bad/total_bad)
        iv_all.append(ival)
        

    iv = sum(iv_all)

    return iv

    
int_accept2 = pd.DataFrame(df_accept,columns=['income','COUNT_ACTIVE_CREDIT_NO_CC',
                                             'CNT_MNTH_FROM_LAST_PMNT','age', 'experience', 
                                             'inquiry_14_day','inquiry_21_day','count_mnth_act_passport','target'])
IV = []

IV.append(get_IV(int_accept2, 'income', 'target'))
IV.append(get_IV(int_accept2, 'COUNT_ACTIVE_CREDIT_NO_CC', 'target'))
IV.append(get_IV(int_accept2, 'CNT_MNTH_FROM_LAST_PMNT', 'target'))
IV.append(get_IV(int_accept2, 'age', 'target'))
IV.append(get_IV(int_accept2, 'experience', 'target'))
IV.append(get_IV(int_accept2, 'inquiry_14_day', 'target'))
IV.append(get_IV(int_accept2, 'inquiry_21_day', 'target'))
IV.append(get_IV(int_accept2, 'count_mnth_act_passport', 'target'))

The following table shows information for each interval variable in Accept.scv:

In [ ]:
data = {'Names': col_names,
        'None_values%': percent_non,
        'Median': median,
        'Mean': mean,
        'Std': std,
        'Num_Outliers': outliers,
        'IV': IV
       } 
int_info = pd.DataFrame(data,columns=['Names','None_values%','Median','Mean','Std','Num_Outliers','IV'])
int_info

### 2.2 Reject

Now, I will redo the same steps for Reject.csv but for now I cannot count IV.

In [ ]:
int_reject1 = pd.DataFrame(df_reject,columns=['income','COUNT_ACTIVE_CREDIT_NO_CC',
                                             'CNT_MNTH_FROM_LAST_PMNT','age', 'experience', 
                                             'inquiry_14_day','inquiry_21_day','count_mnth_act_passport'])
int_reject = int_reject1

In [ ]:
int_reject['CNT_MNTH_FROM_LAST_PMNT'] = int_reject['CNT_MNTH_FROM_LAST_PMNT'].fillna(int_reject["CNT_MNTH_FROM_LAST_PMNT"].mean())
int_reject['experience'] = int_reject['experience'].fillna(int_reject["experience"].mean())
int_reject['count_mnth_act_passport'] = int_reject['count_mnth_act_passport'].fillna(int_reject["count_mnth_act_passport"].mean())
int_reject['age'] = int_reject['age'].fillna(int_reject["age"].mean())

int_reject['income'] = int_reject['income'].fillna(int_reject["income"].median())
int_reject['COUNT_ACTIVE_CREDIT_NO_CC'] = int_reject['COUNT_ACTIVE_CREDIT_NO_CC'].fillna(int_reject["COUNT_ACTIVE_CREDIT_NO_CC"].median())
int_reject['inquiry_14_day'] = int_reject['inquiry_14_day'].fillna(int_reject["inquiry_14_day"].median())
int_reject['inquiry_21_day'] = int_reject['inquiry_21_day'].fillna(int_reject["inquiry_21_day"].median())

if(int_reject.isnull().values.any() == False):
    print("No more Nans")

In [ ]:
percent_non = []
percent_exst = []
count_none = []
col_names = []
for col in int_reject1:
    col_names.append(col)
    percent_non.append((df_reject[col].isnull().sum())/(len(df_reject)/100))
    count_none.append(df_reject[col].isnull().sum())
    

median = []
mean = []
std = []
outliers = []


def z_score_method(df, variable_name):
    columns = df.columns
    z = np.abs(stats.zscore(df))
    threshold = 3
    outlier = []
    index=0
    for item in range(len(columns)):
        if columns[item] == variable_name:
            index = item
    for i, v in enumerate(z[:, index]):
        if v > threshold:
            outlier.append(i)
        else:
            continue
    return outlier
    
for col in int_reject:
    median.append(int_reject[col].median())
    mean.append(int_reject[col].mean())
    std.append(int_reject[col].std())
    outliers.append(len(z_score_method(int_reject, col)))
    
data = {'Names': col_names,
        'None_values%': percent_non,
        'Median': median,
        'Mean': mean,
        'Std': std,
        'Num_Outliers': outliers
       } 
int_info = pd.DataFrame(data,columns=['Names','None_values%','Median','Mean','Std','Num_Outliers'])
int_info

## 3. Необходимо рассчитать для всех категориальных переменных следующее:
  - Мода
  - Доля пропущенных значений
  - Information Value
  - Есть ли выбросы, аномальные значений

### 3.1 Accept

There are 5 categorical variables: 

'INCOME_TYPE','EDUCATION_','CUSTOMER_CATEGORY_','DEPENDANT_NUMBER', 'IS_EMPL'

NaNs I will replace with the most popular category.

To deal with categorical variables I will use Label Encoding for variables as categorical features are ordinal.


In [ ]:
cat_accept1 = pd.DataFrame(df_accept,columns=['INCOME_TYPE','EDUCATION_','CUSTOMER_CATEGORY_',
                                             'DEPENDANT_NUMBER', 'IS_EMPL','target'])

cat_accept = cat_accept1

In [ ]:
cat_accept['INCOME_TYPE'] = cat_accept['INCOME_TYPE'].fillna(cat_accept.mode()['INCOME_TYPE'][0])
cat_accept['EDUCATION_'] = cat_accept['EDUCATION_'].fillna(cat_accept.mode()['EDUCATION_'][0])
cat_accept['CUSTOMER_CATEGORY_'] = cat_accept['CUSTOMER_CATEGORY_'].fillna(cat_accept.mode()['CUSTOMER_CATEGORY_'][0])
cat_accept['DEPENDANT_NUMBER'] = cat_accept['DEPENDANT_NUMBER'].fillna(cat_accept.mode()['DEPENDANT_NUMBER'][0])
cat_accept['IS_EMPL'] = cat_accept['IS_EMPL'].fillna(cat_accept.mode()['IS_EMPL'][0])

if(cat_accept.isnull().values.any() == False):
    print("No more Nans")

In [ ]:
percent_non = []
col_names = []


for col in cat_accept1:
    col_names.append(col)
    percent_non.append((df_accept[col].isnull().sum())/(len(df_accept)/100))
    

mode = []

for col in cat_accept:
    mode.append(cat_accept.mode()[col][0])

In [ ]:
cat_accept2 = cat_accept
cat_accept["EDUCATION_"].replace({"beginner": 0, "elementary": 1, "high": 2, "advanced": 3}, inplace=True)
cat_accept["INCOME_TYPE"].replace({"2NDFL": 0, "OTHER": 1}, inplace=True)
cat_accept["CUSTOMER_CATEGORY_"].replace({"Corporate": 0, "VIP": 1}, inplace=True)

The following function calculates IV for each varible using WOE. The method of calculating WOE:

I separated data in bins, one bin for each category and calculate WOE value for each bin;

Then the function calculates IV for each bin and sums up values to get IV for the variable

In [ ]:
def get_IV_cat(df, feature, target):
    lst = []
    unique_values = df[feature].unique()
    for val in unique_values:
        lst.append([feature,                                                                                                                   
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good 
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]   # Bad  
                   ])

 
    data = pd.DataFrame(lst, columns=['Variable', 'Good', 'Bad'])

        
    total_bad = df[df[target] == 1].count()[feature]
    total_good = df.shape[0] - total_bad
    
    data['Distribution Good'] = data['Good']/ total_good
    data['Distribution Bad'] = data['Bad'] / total_bad
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])
    

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])
    data.index = range(len(data.index))

    iv = data['IV'].sum()

    return iv
    
IV = []

IV.append(get_IV_cat(cat_accept, 'INCOME_TYPE', 'target'))
IV.append(get_IV_cat(cat_accept, 'EDUCATION_', 'target'))
IV.append(get_IV_cat(cat_accept, 'CUSTOMER_CATEGORY_', 'target'))
IV.append(get_IV_cat(cat_accept, 'DEPENDANT_NUMBER', 'target'))
IV.append(get_IV_cat(cat_accept, 'IS_EMPL', 'target'))
IV.append(0)

In [ ]:
data = {'Names': col_names,
        'None_values%': percent_non,
        'Mode': mode,
        'IV': IV
       } 
int_info = pd.DataFrame(data,columns=['Names','None_values%','Mode','IV'])
int_info = int_info.drop([5])
int_info


To look if there are any anomaloies I will construct histogram for each variable.

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15,5))

axes[0].hist(x = 'INCOME_TYPE', data = cat_accept)
axes[0].set_xlabel("INCOME_TYPE ")
axes[1].hist(x = 'EDUCATION_', data = cat_accept)
axes[1].set_xlabel("EDUCATION_ ")
axes[2].hist(x = 'CUSTOMER_CATEGORY_', data = cat_accept)
axes[2].set_xlabel("CUSTOMER_CATEGORY_ ")
axes[3].hist(x = 'DEPENDANT_NUMBER', data = cat_accept)
axes[3].set_xlabel("DEPENDANT_NUMBER ")
axes[4].hist(x = 'IS_EMPL', data = cat_accept)
axes[4].set_xlabel("IS_EMPL ")


plt.show()

So, I can say that there is no anomalies. 

### 3.2 Reject

Now, I will do the same but for reject.csv. Except for IV.

In [ ]:
cat_reject1 = pd.DataFrame(df_reject,columns=['INCOME_TYPE','EDUCATION_','CUSTOMER_CATEGORY_',
                                             'DEPENDANT_NUMBER', 'IS_EMPL','target'])

cat_reject = cat_reject1

In [ ]:
cat_reject['INCOME_TYPE'] = cat_reject['INCOME_TYPE'].fillna(cat_reject.mode()['INCOME_TYPE'][0])
cat_reject['EDUCATION_'] = cat_reject['EDUCATION_'].fillna(cat_reject.mode()['EDUCATION_'][0])
cat_reject['CUSTOMER_CATEGORY_'] = cat_reject['CUSTOMER_CATEGORY_'].fillna(cat_reject.mode()['CUSTOMER_CATEGORY_'][0])
cat_reject['DEPENDANT_NUMBER'] = cat_reject['DEPENDANT_NUMBER'].fillna(cat_reject.mode()['DEPENDANT_NUMBER'][0])
cat_reject['IS_EMPL'] = cat_reject['IS_EMPL'].fillna(cat_reject.mode()['IS_EMPL'][0])

if(cat_reject.isnull().values.any() == False):
    print("No more Nans")

In [ ]:
percent_non = []
col_names = []


for col in cat_reject1:
    col_names.append(col)
    if(col != 'target'):
        percent_non.append((df_reject[col].isnull().sum())/(len(df_reject)/100))
    else: 
        percent_non.append(0)
    

mode = []

for col in cat_reject:
    mode.append(cat_reject.mode()[col][0])

In [ ]:
cat_reject2 = cat_reject
cat_reject["EDUCATION_"].replace({"beginner": 0, "elementary": 1, "high": 2, "advanced": 3}, inplace=True)
cat_reject["INCOME_TYPE"].replace({"2NDFL": 0, "OTHER": 1}, inplace=True)
cat_reject["CUSTOMER_CATEGORY_"].replace({"Corporate": 0, "VIP": 1}, inplace=True)

In [ ]:
data = {'Names': col_names,
        'None_values%': percent_non,
        'Mode': mode
       } 
int_info = pd.DataFrame(data,columns=['Names','None_values%','Mode'])
int_info = int_info.drop([5])
int_info


Checking anomalies:

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15,5))

axes[0].hist(x = 'INCOME_TYPE', data = cat_reject)
axes[0].set_xlabel("INCOME_TYPE ")
axes[1].hist(x = 'EDUCATION_', data = cat_reject)
axes[1].set_xlabel("EDUCATION_ ")
axes[2].hist(x = 'CUSTOMER_CATEGORY_', data = cat_reject)
axes[2].set_xlabel("CUSTOMER_CATEGORY_ ")
axes[3].hist(x = 'DEPENDANT_NUMBER', data = cat_reject)
axes[3].set_xlabel("DEPENDANT_NUMBER ")
axes[4].hist(x = 'IS_EMPL', data = cat_reject)
axes[4].set_xlabel("IS_EMPL ")


plt.show()

No anomalies

## 4. Построить логистическую регрессию только на одобренных заявках с преобразованными переменными WoE. Какое значение GINI? F1 мера?

Firstly I will create WOE variables for each variable, using the same functions as before with small corrections in order to calculate columns for the new WOE variables:

In [ ]:
all_accept = pd.concat([int_accept, cat_accept], axis=1)

In [ ]:
def get_IV(df, feature, target):
    lst = []
    bins = df[feature].quantile(np.linspace(0, 1, 21)).values
    bin_index = range(1, len(bins))
    for index in bin_index:
        val = df[(df[feature] >= bins[index-1])&(df[feature] <= bins[index])]
        bin_low = bins[index-1]
        bin_high = bins[index]
        lst.append([feature,
                    bin_low,
                    bin_high,
                    val[(val[target] == 0)].count()[feature],  # Good 
                    val[(val[target] == 1)].count()[feature]   # Bad  
                   ])
 
    data = pd.DataFrame(lst, columns=['Variable', 'Low', 'High', 'Good', 'Bad'])

        
    total_bad = df[df[target] == 1].count()[feature]
    total_good = df.shape[0] - total_bad
    
    data['Distribution Good'] = data['Good']/ total_good
    data['Distribution Bad'] = data['Bad'] / total_bad
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    data = data.sort_values(by=['WoE'], ascending=[True])
    data.index = range(len(data.index))
    
    data1 = data
    
        
    #Combine bins with similar WOE values
    n = range(1, len(data1))
    for i in n:
        if (data1['WoE'][i] - data1['WoE'][i-1] < 0.08):
            data.loc[i,'WoE'] = data1['WoE'][i-1]

    data = data.sort_values(by=['Low'], ascending=[True])

    return data

In [ ]:
def get_IV_cat(df, feature, target):
    lst = []
    unique_values = df[feature].unique()
    for val in unique_values:
        bin_val = val
        lst.append([feature, 
                    val,
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good 
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]   # Bad  
                   ])

 
    data = pd.DataFrame(lst, columns=['Variable','Bin_val' ,'Good', 'Bad'])

        
    total_bad = df[df[target] == 1].count()[feature]
    total_good = df.shape[0] - total_bad
    
    data['Distribution Good'] = data['Good']/ total_good
    data['Distribution Bad'] = data['Bad'] / total_bad
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])
    

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])
    data = data.sort_values(by=['WoE'], ascending=[True])
    data.index = range(len(data.index))

    iv = data['IV'].sum()

    return data

In [ ]:
def make_woe_cat(df, feature, target):
    data = get_IV_cat(df, feature, target)
    for i in range(0, len(data)):      
        df.loc[(df[feature] == data['Bin_val'][i]), feature +'WOE'] = data['WoE'][i] 

In [ ]:
make_woe_cat(all_accept, 'INCOME_TYPE', 'target')
make_woe_cat(all_accept, 'EDUCATION_', 'target')
make_woe_cat(all_accept, 'CUSTOMER_CATEGORY_', 'target')
make_woe_cat(all_accept, 'DEPENDANT_NUMBER', 'target')
make_woe_cat(all_accept, 'IS_EMPL', 'target')

for col in int_accept:
    data = get_IV(all_accept, col, 'target')
    for i in range(0, len(data)): 
        all_accept.loc[(all_accept[col] >= data['Low'][i])&(all_accept[col] <= data['High'][i]), col +'WOE'] = data['WoE'][i]

In [ ]:
all_accept.T

Now, I will build logistic regression on obtained WOE variables:

I will split data set into a training set and a validation set and then fit a logistic regression model using only the training observations

In [ ]:
X = all_accept[['INCOME_TYPEWOE', 'EDUCATION_WOE', 'CUSTOMER_CATEGORY_WOE', 'DEPENDANT_NUMBERWOE', 'IS_EMPLWOE',
               'incomeWOE', 'COUNT_ACTIVE_CREDIT_NO_CCWOE', 'CNT_MNTH_FROM_LAST_PMNTWOE', 'ageWOE', 'experienceWOE',
               'inquiry_14_dayWOE', 'inquiry_21_dayWOE', 'count_mnth_act_passportWOE']]

y = all_accept['target']
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.33, random_state=0)

logreg_accept = LogisticRegression()
logreg_accept.fit(X_train, y_train)

logit1 = sm.Logit(y,sm.add_constant(X)).fit()

roc = 0.5
thresh = 0.5

for t in np.linspace(0, 1, 1000):
    predict = np.where(logreg_accept.predict_proba(X_train)[:,1] > t, 1, 0) #prob of 1
    roc_t = roc_auc_score(y_train, predict)
    if roc_t > roc:
        roc = roc_t
        best_thresh = t
        
predict = np.where(logreg_accept.predict_proba(X_test)[:,1] > best_thresh, 1, 0)
print("GINI:", 2*roc_auc_score(y_test, predict) - 1)
print("F1 Score:", f1_score(y_test, predict))

## 5. Провести анализ Reject Inference. Какая доля отказанных заявок? 

Firstly, I will create WOE variables foe each variable for the reject.csv by using WOE from accept.csv. For the values from reject.csv that are larger than the max bin or lower than the min bin from accept.csv I will give WOE of the highest and the lowest bins.

In [ ]:
all_reject = pd.concat([int_reject, cat_reject], axis=1)

In [ ]:
def make_woe_cat_r(df_acc, df_rej, feature, target):
    data = get_IV_cat(df_acc, feature, target)
    for i in range(0, len(data)):      
        df_rej.loc[(df_rej[feature] == data['Bin_val'][i]), feature +'WOE'] = data['WoE'][i] 

make_woe_cat_r(all_accept, all_reject, 'INCOME_TYPE', 'target')
make_woe_cat_r(all_accept, all_reject, 'EDUCATION_', 'target')
make_woe_cat_r(all_accept, all_reject, 'CUSTOMER_CATEGORY_', 'target')
make_woe_cat_r(all_accept, all_reject, 'DEPENDANT_NUMBER', 'target')
make_woe_cat_r(all_accept, all_reject, 'IS_EMPL', 'target')

for col in int_reject:
    data = get_IV(all_accept, col, 'target')
    data.loc[0,'Low'] = all_reject[col].min() 
    data.loc[19,'High'] = all_reject[col].max() 
    for i in range(0, len(data)): 
        all_reject.loc[(all_reject[col] >= data['Low'][i])&(all_reject[col] <= data['High'][i]), col +'WOE'] = data['WoE'][i]

In [ ]:
all_reject.T

Now, I will use my logistic regression model from accept.csv in order to find predictions for 'target' for reject.csv:

In [ ]:
Xr = all_reject[['INCOME_TYPEWOE', 'EDUCATION_WOE', 'CUSTOMER_CATEGORY_WOE', 'DEPENDANT_NUMBERWOE', 'IS_EMPLWOE',
               'incomeWOE', 'COUNT_ACTIVE_CREDIT_NO_CCWOE', 'CNT_MNTH_FROM_LAST_PMNTWOE', 'ageWOE', 'experienceWOE',
               'inquiry_14_dayWOE', 'inquiry_21_dayWOE', 'count_mnth_act_passportWOE']]

        
predict = np.where(logreg_accept.predict_proba(Xr)[:,1] > best_thresh, 1, 0)
all_reject['target'] = predict

Calculate share of rejected applications:

In [ ]:
len(all_reject) / (len(all_reject) + len(all_accept))

## 6. Построить логистическую регрессию на всех заявках с преобразованными переменными WoE. Какое значение GINI, F1? Изменилась ли модель?

Combine two tables:

In [ ]:
all_app = pd.concat([all_accept, all_reject]).reset_index(drop=True)

Now, build logistic regression model on all_app and calculate GINI and F1:

In [ ]:
X = all_app[['INCOME_TYPEWOE', 'EDUCATION_WOE', 'CUSTOMER_CATEGORY_WOE', 'DEPENDANT_NUMBERWOE', 'IS_EMPLWOE',
               'incomeWOE', 'COUNT_ACTIVE_CREDIT_NO_CCWOE', 'CNT_MNTH_FROM_LAST_PMNTWOE', 'ageWOE', 'experienceWOE',
               'inquiry_14_dayWOE', 'inquiry_21_dayWOE', 'count_mnth_act_passportWOE']]

y = all_app['target']
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.33, random_state=0)

logreg_accept = LogisticRegression()
logreg_accept.fit(X_train, y_train)

logit2 = sm.Logit(y,sm.add_constant(X)).fit()

roc = 0.5
thresh = 0.5

for t in np.linspace(0, 1, 1000):
    predict = np.where(logreg_accept.predict_proba(X_train)[:,1] > t, 1, 0) #prob of 1
    roc_t = roc_auc_score(y_train, predict)
    if roc_t > roc:
        roc = roc_t
        best_thresh = t
        
predict = np.where(logreg_accept.predict_proba(X_test)[:,1] > best_thresh, 1, 0)
print("GINI:", 2*roc_auc_score(y_test, predict) - 1)
print("F1 Score:", f1_score(y_test, predict))

We can see that both GINI coefficient and F1 score increased => the model with all applictions has a better performace. 

In [ ]:
print("1st Model", logit1.summary())

In [ ]:
print("2nd Model", logit2.summary())

We can see that $R^2$ statistic is also higher for the second model. And we can observe changes in the sign of some coefficients and also change in the significance of some variables.

## 7. Какую модель вы рекомендуете для внедрения в продуктивную среду?

   First of all we can see that such measures of a model's accuracy as GINI coefficient, F1 score and R-squared are all higher for the second model. Moreover, the seconed model was trained on the larger amount of data, which means that it increases the chance of overfitting and has more balanced classes. 

   However, the second model can be risky: if the model misclassified a large amount of "bad" clients as "good", this would have a negative impact on the accuracy of predictions and using this model may lead to giving credits to undesirable clients who will have debt problems. On the other hand, as we cannot say exactly if they were classified correctly or not, there can be a situation where we lost potential clients and they were classified to reject.csv and using reject inference helped to find such clients and therefore adding more "good" clients to the training set increases the accuracy. 
   
   Overall, in my example I would recommend to use the second model, due to the reason that first model has very low measures of a model's accuracy. In general, I think that model built by second method will also be preferable, but for a better performance, it would be helpful to decrease the risk of misclassification for the reject.csv clients, for example by adding restrictions on what clients are definitely undesirable. 